In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd
from scipy import stats


from gurobi_implementation import GurobiSolution
from data_market import Seller, MarketOperator, Market, Buyer
from p2p_gurobi import Agents, FirstStageMarket

In [2]:
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [3]:
connection_matrix = [[0, 1, 1, 1, 1],
                    [1, 0, 1, 1, 1],
                    [1, 1, 0, 1, 1],
                    [1, 1, 1, 0, 1],
                    [1, 1, 1, 1, 0]]

kappa = [[0, 10, 10, 10, 10],
        [10, 0, 10, 10, 10],
        [10, 10, 0, 10, 10],
        [10, 10, 10, 0, 10],
        [10, 10, 10, 10, 0]]                   

In [4]:
#model_1
actual_generations = [3, 4, 5, 4, 3]
demand = [6, 7, 8, 9, 10]

price_da_buy = 3
price_da_sell = 2
price_rt_buy = 4
price_rt_sell = 1

base_forecasts = [stats.norm(loc = generation, scale = 1.0) for generation in actual_generations]

agents_forecasts = []
for base_forecast in base_forecasts:
    probas, values = np.histogram(base_forecasts[0].rvs(100000), bins = 100, density = True)
    probas = probas / probas.sum()
    agents_forecasts.append((probas, values))

agents = []
for id, actual_generation in enumerate(actual_generations):
    agent = Agents(id = id,
                probabilities = agents_forecasts[id][0],
                generation_values = agents_forecasts[id][1],
                demand = demand[id],
                connections = connection_matrix[id],
                kappa = kappa[id])

    agents.append(agent)

In [5]:
model_1 = gp.Model()

In [6]:
market_1 = FirstStageMarket(agents= agents,
                            model = model_1,
                            price_da_buy = price_da_buy,
                            price_da_sell = price_da_sell,
                            price_rt_buy = price_rt_buy,
                            price_rt_sell = price_rt_sell)

In [7]:
market_1.build_model()

In [8]:
model_1.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 525 rows, 1035 columns and 2565 nonzeros
Model fingerprint: 0xb4254e86
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 3e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [3e-02, 1e+01]
Presolve removed 35 rows and 50 columns
Presolve time: 0.00s
Presolved: 490 rows, 985 columns, 2440 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     786    8.1139457e+01   0.000000e+00   0.000000e+00      0s

Solved in 786 iterations and 0.01 seconds (0.01 work units)
Optimal objective  8.113945691e+01


In [9]:
def second_stage_resolution(model, agents, actual_generations):
    second_stage_decisions = []
    actual_costs = []
    for agent in agents:
        second_stage_decision = (agent.demand 
                                - actual_generations[agent.id] 
                                - model.getVarByName(f'Agent {agent.id} day-ahead purchase').X
                                + model.getVarByName(f'Agent {agent.id} day-ahead sale').X
                                - model.getVarByName(f'Agent {agent.id} net trading').X)

        second_stage_cost = price_rt_buy * second_stage_decision if second_stage_decision > 0 else price_rt_sell * second_stage_decision

        second_stage_decisions.append(second_stage_decision)

        actual_cost = (price_da_buy * model.getVarByName(f'Agent {agent.id} day-ahead purchase').X 
                    - price_da_sell * model.getVarByName(f'Agent {agent.id} day-ahead sale').X
                    + second_stage_cost )


        actual_costs.append(actual_cost)

    return second_stage_decisions, actual_costs
        

In [10]:
second_stage_resolution(model_1, agents, actual_generations)

([0.36878395342717774,
  -0.6302837319265411,
  -1.589484770976938,
  -0.6365429267929299,
  0.37383815928408115],
 [1.475135813708711,
  -0.6302837319265411,
  67.75158317997851,
  -0.6365429267929299,
  1.4953526371363246])

In [11]:
second_stage_resolution(model_1, agents, actual_generations)

([0.36878395342717774,
  -0.6302837319265411,
  -1.589484770976938,
  -0.6365429267929299,
  0.37383815928408115],
 [1.475135813708711,
  -0.6302837319265411,
  67.75158317997851,
  -0.6365429267929299,
  1.4953526371363246])

In [12]:
sum(second_stage_resolution(model_1, agents, actual_generations)[1])

69.45524497210407

In [13]:
#model_2
actual_generations = [3, 4, 5, 4, 3]
demand = [6, 7, 8, 9, 10]

price_da_buy = 3
price_da_sell = 2
price_rt_buy = 4
price_rt_sell = 1

base_forecasts = [stats.norm(loc = generation, scale = 0.5) for generation in actual_generations]

agents_forecasts = []
for base_forecast in base_forecasts:
    probas, values = np.histogram(base_forecasts[0].rvs(100000), bins = 100, density = True)
    probas = probas / probas.sum()
    agents_forecasts.append((probas, values))

agents = []
for id, actual_generation in enumerate(actual_generations):
    agent = Agents(id = id,
                probabilities = agents_forecasts[id][0],
                generation_values = agents_forecasts[id][1],
                demand = demand[id],
                connections = connection_matrix[id],
                kappa = kappa[id])

    agents.append(agent)

In [14]:
model_2 = gp.Model()

market_2 = FirstStageMarket(agents= agents,
                            model = model_2,
                            price_da_buy = price_da_buy,
                            price_da_sell = price_da_sell,
                            price_rt_buy = price_rt_buy,
                            price_rt_sell = price_rt_sell)

In [15]:
market_2.build_model()

In [16]:
model_2.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 525 rows, 1035 columns and 2565 nonzeros
Model fingerprint: 0xe3cc3853
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 3e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [9e-01, 9e+00]
Presolve removed 37 rows and 54 columns
Presolve time: 0.00s
Presolved: 488 rows, 981 columns, 2430 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     728    7.8027231e+01   0.000000e+00   0.000000e+00      0s

Solved in 728 iterations and 0.01 seconds (0.01 work units)
Optimal objective  7.802723128e+01


In [17]:
second_stage_resolution(model_2, agents, actual_generations)

([0.1875959328993826,
  -0.8237991814494707,
  -1.798650312400346,
  -0.8045350169824541,
  0.1911091369606197],
 [0.7503837315975304,
  -0.8237991814494707,
  -1.798650312400346,
  -0.8045350169824541,
  72.90927487075928])

In [18]:
sum(second_stage_resolution(model_2, agents, actual_generations)[1])

70.23267409152453

In [19]:
#model_3
actual_generations = [3, 4, 5, 4, 3]
demand = [6, 7, 8, 9, 10]

price_da_buy = 3
price_da_sell = 2
price_rt_buy = 4
price_rt_sell = 1

base_forecasts = [stats.norm(loc = generation, scale = 0.1) for generation in actual_generations]

agents_forecasts = []
for base_forecast in base_forecasts:
    probas, values = np.histogram(base_forecasts[0].rvs(100000), bins = 100, density = True)
    probas = probas / probas.sum()
    agents_forecasts.append((probas, values))

agents = []
for id, actual_generation in enumerate(actual_generations):
    agent = Agents(id = id,
                probabilities = agents_forecasts[id][0],
                generation_values = agents_forecasts[id][1],
                demand = demand[id],
                connections = connection_matrix[id],
                kappa = kappa[id])

    agents.append(agent)

In [20]:
model_3 = gp.Model()

market_3 = FirstStageMarket(agents= agents,
                            model = model_3,
                            price_da_buy = price_da_buy,
                            price_da_sell = price_da_sell,
                            price_rt_buy = price_rt_buy,
                            price_rt_sell = price_rt_sell)

In [21]:
market_3.build_model()
model_3.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 525 rows, 1035 columns and 2565 nonzeros
Model fingerprint: 0x6060944f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 3e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [3e+00, 7e+00]
Presolve removed 36 rows and 52 columns
Presolve time: 0.00s
Presolved: 489 rows, 983 columns, 2435 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     726    7.5611336e+01   0.000000e+00   0.000000e+00      0s

Solved in 726 iterations and 0.01 seconds (0.01 work units)
Optimal objective  7.561133602e+01


In [22]:
second_stage_resolution(model_3, agents, actual_generations)

([0.041914953796013066,
  -0.9614466313843666,
  -1.9635470537497843,
  -0.9652389868577735,
  0.03840344676461438],
 [0.16765981518405226,
  -0.9614466313843666,
  -1.9635470537497843,
  73.46450382743612,
  0.15361378705845752])

In [23]:
sum(second_stage_resolution(model_3, agents, actual_generations)[1])

70.86078374454448

In [25]:
#model_4
actual_generations = [3, 4, 5, 4, 3]
demand = [6, 7, 8, 9, 10]

price_da_buy = 3
price_da_sell = 2
price_rt_buy = 4
price_rt_sell = 1

base_forecasts = [stats.norm(loc = generation, scale = 0.0) for generation in actual_generations]

agents_forecasts = []
for base_forecast in base_forecasts:
    probas, values = np.histogram(base_forecasts[0].rvs(100000), bins = 100, density = True)
    probas = probas / probas.sum()
    agents_forecasts.append((probas, values))

agents = []
for id, actual_generation in enumerate(actual_generations):
    agent = Agents(id = id,
                probabilities = agents_forecasts[id][0],
                generation_values = agents_forecasts[id][1],
                demand = demand[id],
                connections = connection_matrix[id],
                kappa = kappa[id])

    agents.append(agent)

In [26]:
model_4 = gp.Model()

market_4 = FirstStageMarket(agents= agents,
                            model = model_4,
                            price_da_buy = price_da_buy,
                            price_da_sell = price_da_sell,
                            price_rt_buy = price_rt_buy,
                            price_rt_sell = price_rt_sell)

market_4.build_model()
model_4.optimize()                            

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 525 rows, 1035 columns and 2565 nonzeros
Model fingerprint: 0x70753b3d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [3e+00, 8e+00]
Presolve removed 515 rows and 1005 columns
Presolve time: 0.00s
Presolved: 10 rows, 30 columns, 70 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5000000e+02   4.074500e+01   0.000000e+00      0s
       7    7.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.500000000e+01


In [27]:
second_stage_resolution(model_4, agents, actual_generations)

([0.0, -1.0, -2.0, -1.0, 0.0], [36.0, -1.0, -2.0, 17.0, 21.0])

In [29]:
sum(second_stage_resolution(model_4, agents, actual_generations)[1])

71.0

In [39]:
print('Model_1 net trades:', [model_1.getVarByName(f'Agent {id} net trading').X for id, _ in enumerate(agents)])
print('Model_2 net trades:', [model_2.getVarByName(f'Agent {id} net trading').X for id, _ in enumerate(agents)])
print('Model_3 net trades:', [model_3.getVarByName(f'Agent {id} net trading').X for id, _ in enumerate(agents)])
print('Model_4 net trades:', [model_4.getVarByName(f'Agent {id} net trading').X for id, _ in enumerate(agents)])

Model_1 net trades: [2.6312160465728223, 3.630283731926541, -18.524204546008214, 5.63654292679293, 6.626161840715919]
Model_2 net trades: [2.8124040671006174, 3.8237991814494707, 4.798650312400346, 5.804535016982454, -17.239388577932885]
Model_3 net trades: [2.958085046203987, 3.9614466313843666, 4.963547053749784, -18.844675284573523, 6.961596553235386]
Model_4 net trades: [-9.0, 4.0, 5.0, 0.0, 0.0]
